To run this notebook in colab, kindly upload the data provided as data.zip, which can be found in the link 

Post this upload your test-set as test.csv which should have the format

Sequence

((A+C+T+G)\*\n)\* // each sequence should be 81 nucleotides long

A sample file is provided in the drive link. 

In [1]:
!unzip data.zip

unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.


In [ ]:
import keras
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.preprocessing import scale
from itertools import product
from keras.layers import Dense, Embedding, LSTM, Dropout, MaxPool1D, Conv1D, Flatten, Conv2D, Concatenate, Add, BatchNormalization, AvgPool1D
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:

# code borrowed from iPromoter-BnCNN source code starts

new_name = 'test_data.csv'
sequences = pd.read_csv('test_data.csv')

seqs = []
for index,row in sequences.iterrows():
	sequence = (row['Sequence'])
	seqs.append(sequence.upper())
main_seqs = np.copy(seqs)
predictions_dictionary = {}

# Structural Properties of Di Nucleotide
di_prop = pd.read_csv('data/DNA_Di_Prop.txt')
di_prop = di_prop.iloc[:, 1:]
scaled_di_prop = scale(di_prop, axis=1) # Standardization
di_cols = di_prop.columns.tolist()
di_prop = pd.DataFrame(scaled_di_prop, columns=di_cols)
pp_di = {}
for i in range(16):
	key = di_prop.columns[i]
	items = di_prop.iloc[:, i].tolist()
	pp_di[key] = items
# Structural Properties of tri nucleotide
tri_prop = pd.read_csv('data/DNA_Tri_Prop.txt')
tri_prop = tri_prop.iloc[:, 1:]
scaled_tri_prop = scale(tri_prop, axis=1) # Standardization
tri_cols = tri_prop.columns.tolist()
tri_prop = pd.DataFrame(scaled_tri_prop, columns=tri_cols)
pp_tri = {}
for i in range(64):
	key = tri_prop.columns[i]
	items = tri_prop.iloc[:, i].tolist()
	pp_tri[key] = items


# Mono-mer Feature
X1 = np.empty((len(seqs), len(seqs[0])))
alphabet = 'ATGC'
for i in range(len(seqs)):
	for j in range(len(seqs[0])):
		X1[i, j] = next((k for k, letter in enumerate(alphabet) if letter == seqs[i][j]))
	
X1 = to_categorical(X1)


# Tri-mer Feature
lookup_table = []
for p in product('ATGC', repeat=3):
	w = ''.join(p)
	lookup_table.append(w)
X2 = np.empty((len(seqs), len(seqs[0])-2))
for i in range(len(seqs)):
	for j in range(len(seqs[0])-2):
		w = seqs[i][j:j+3]
		X2[i,j] = lookup_table.index(w)      
X2 = to_categorical(X2)


# Di nucleotide properties feature
X3 = np.empty([len(seqs), 80, 90], dtype=float)
for i in range(len(seqs)):
	for j in range(80):
		word = seqs[i][j:j+2]
		value = pp_di[word]
		for k in range(90):
			X3[i, j, k] = value[k]


# Tri nucleotide properties feature
X4 = np.empty([len(seqs), 79, 12], dtype=float)
for i in range(len(seqs)):
	for j in range(79):
		word = seqs[i][j:j+3]
		value = pp_tri[word]
		for k in range(12):
			X4[i, j, k] = value[k]

# code borrowed from iPromoter-BnCNN source code ends
# For debugging
# print(X1.shape)

In [ ]:
%cd data/models/
model1 = keras.models.load_model("model1.h5")
model2 = keras.models.load_model("model2.h5")
model3 = keras.models.load_model("model3.h5")
model4 = keras.models.load_model("model4.h5")
%cd ../../

In [ ]:
A1 = model1.predict([X1,X2,X3,X4])
A2 = model2.predict([X1,X2,X3,X4])
A3 = model3.predict([X1,X2,X3,X4])
A4 = model4.predict([X1,X2,X3,X4])

B1= np.where(A1[:,0]>A1[:,1], 1, 0)
B2= np.where(A2[:,0]>A2[:,1], 1, 0)
B3= np.where(A3[:,0]>A3[:,1], 1, 0)
B4= np.where(A4[:,0]>A4[:,1], 1, 0)
pred =[]
for i in range(len(B1)):
    if(B1[i]==1):
        pred.append("Sigma 70")
    elif(B1[i]==0 and B2[i]==1):
        pred.append("Sigma 24")
    elif(B1[i]==0 and B2[i]==0 and B3[i]==1):
        pred.append("Sigma 32")
    elif(B1[i]==0 and B2[i]==0 and B3[i]==0 and B4[i]==1):
        pred.append("Sigma 28")
    elif(B1[i]==0 and B2[i]==0 and B3[i]==0 and B4[i]==0):
        pred.append("Sigma 38")

print(pred)